# Optimize Search Query

In [14]:
# pip install sentence-transformers

### 1) Read Webscraped Data 

Convert the webscraped JSON data into panda dataframe

In [1]:
import json
import pandas as pd

def load_json_file(filename):
    try:
        with open(filename, 'r',encoding='utf-8') as f:
            data = json.load(f)
            return data
    except json.JSONDecodeError as e:
        print(f"Error reading {filename}: {e}")
        with open(filename, 'r',encoding='utf-8') as f:
            data=[]
            for line in f:
                d = json.loads(line)
                data.append(d)
            return data
data = load_json_file('Ecocheck_Data.json') 
df = pd.DataFrame(data)

### 2) Considering Data having True Labels

In [2]:
true_df = df[df['labels'].apply(lambda x: len(x) > 0)]

### 3) Cleaning the columns [price, organic_label, vegan_label]

In [3]:
'''                          Convert the categorical data into numeric of columns organic_label & vegan_label
'''
# Function to handle different data types and convert lists to strings
def process_labels(label):
    if isinstance(label, list) and label:
        return ' '.join(label)
    elif isinstance(label, str):
        return label.strip()  # Strip any leading/trailing whitespace
    return 'none'  # Return 'none' if label is None or empty list

# Apply the function to the relevant columns using .loc to avoid SettingWithCopyWarning
true_df.loc[:, 'organic_label'] = true_df['organic_label'].apply(process_labels)
true_df.loc[:, 'vegan_label'] = true_df['vegan_label'].apply(process_labels)

# Ensure 'Bio' and 'Vegan' come first by creating custom sort keys
true_df.loc[:, 'organic_label'] = true_df['organic_label'].apply(lambda x: 1 if 'Bio' in x else 0)
true_df.loc[:, 'vegan_label'] = true_df['vegan_label'].apply(lambda x: 1 if 'Vegan' in x else 0)

'''                             Convert the string data into numeric of column price
'''

true_df.loc[:,'price'] = true_df['price'].str.replace('[^\d.]', '', regex=True)  # Remove non-numeric characters except '.'
true_df.loc[:,'price'] = pd.to_numeric(true_df['price'], errors='coerce')  # Convert to numeric, coerce errors to NaN

C:\Users\AK traders\AppData\Local\Temp\ipykernel_3024\451410679.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df.loc[:, 'organic_label'] = true_df['organic_label'].apply(process_labels)
C:\Users\AK traders\AppData\Local\Temp\ipykernel_3024\451410679.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df.loc[:, 'vegan_label'] = true_df['vegan_label'].apply(process_labels)
C:\Users\AK traders\AppData\Local\Temp\ipykernel_3024\451410679.py:16: SettingWithCopyWarning: 
A value is trying to be 

### 4) Assembling input columns into new column

columns_to_combine = ['allergen', 'brandsText', 'categories', 'ingredientsText', 'labels', 'name', 'description']

In [4]:
def combine_columns(row):
    combined = ''
    columns_to_combine = ['allergen', 'brandsText', 'categories', 'ingredientsText', 'labels', 'name', 'description']
    for col in columns_to_combine:
        value = row[col]
        if isinstance(value, list):
            combined += ' '.join(value) + ' '
        else:
            combined += str(value) + ' '
    return combined.strip()  # Remove any trailing whitespace

true_df.loc[:,'combined'] = true_df.apply(combine_columns, axis=1)


C:\Users\AK traders\AppData\Local\Temp\ipykernel_3024\558376629.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_df.loc[:,'combined'] = true_df.apply(combine_columns, axis=1)


### 5) Apply the text embedding

In [5]:
from sentence_transformers import SentenceTransformer

# Load the multilingual model
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

# Extract sentences from the 'combined' column
sentences = true_df['combined'].tolist()

# Generate embeddings
embeddings = model.encode(sentences)

# Check the shape of embeddings
print(embeddings.shape)

ModuleNotFoundError: No module named 'sentence_transformers'

### 6) Indexing

In [21]:
import faiss

def indexing(embeddings, cell_num):
    
    d = embeddings.shape[1]
    nlist = cell_num  # how many cells
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFFlat(quantizer, d, nlist)

    # trained Index
    index.train(embeddings)

    index.add(embeddings)
    index.ntotal  # number of embeddings indexed
    
    return index

index = indexing(embeddings, 100)

In [22]:
index

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7f5987d30480> >

In [75]:
import time

def SearchInputQuery(query, num_of_results, index, nprob, most_relevant_columns):
    start_time = time.time()
    
    queryList = [query]
    xq = model.encode(queryList)   
    k = num_of_results #50 # 4
    index.nprobe = nprob#5
    D, I = index.search(xq, k)  # search
    
    # recommended list
    recommended_products = true_df.iloc[I[0]]

    sort_columns = most_relevant_columns
    if len(sort_columns) > 0:
        if 'organic_label' in sort_columns or 'vegan_label' in sort_columns:
            sorted_df = recommended_products.sort_values(by=sort_columns, ascending=False)
            result = sorted_df.head(5)
        elif 'price' in sort_columns or 'emissionFromProducts' in sort_columns:
            sorted_df = recommended_products.sort_values(by='price', ascending=True)
            result = sorted_df.head(5)

    else:
        result = recommended_products.head(5)

    end_time = time.time()  
    execution_time = end_time - start_time  
   # print(f"Execution time: {execution_time*1e6} micro-seconds")
    return result, execution_time


In [45]:
most_relevant_columns = ['organic_label', 'vegan_label']
search_query = "chicken"

Best_results, running_time = SearchInputQuery(search_query, 50, index, 5, most_relevant_columns)

print(f"Execution time: {running_time*1e3} milli-seconds")
print (f"Search Query: {search_query} \n")
Best_results[['name','labels']]


Execution time: 18.20969581604004 milli-seconds
Search Query: chicken 



,name,labels
6301,Alnatura Bio Hühner Bouillon 6ST 66G,"[Bio, Glutenfree, Palmoel]"
3950,Geflügelsalat,[Bio]
6776,Little Lunch Bio Geflügel Knochen-Fond 400ML,[Bio]
3932,Hühner-Nudel-Topf,"[Bio, Non-Vegetarian, Vegetarian]"
11394,EDEKA Naturals Rind & Huhn 125G,[Bio]


In [87]:
most_relevant_columns = ['organic_label', 'vegan_label']
search_query = "biyo süt"

Best_results, running_time = SearchInputQuery(search_query, 50, index, 5, most_relevant_columns)

print(f"Execution time: {running_time*1e3} milli-seconds")
print (f"Search Query: {search_query} \n")
Best_results[['name','labels']]


Execution time: 26.424884796142578 milli-seconds
Search Query: biyo süt 



,name,labels
16614,"Schwarzwaldmilch Bio lactosefreie H-Milch 3,5%...","[Bio, Vegetarian, Vegan]"
17426,"Schwarzwaldmilch Bio lactosefreie H-Milch 1,5%...","[Bio, Vegetarian, Vegan]"
18338,"Aptamil Anti-Reflux Komplettnahrung 800 g, 4er...","[Bio, Vegetarian, Vegan]"
3786,Haferdrink-Cappuccino,"[Bio, Vegan]"
18184,"Aptamil Anfangsmilch Pre 800 g, 6er Pack","[Bio, Vegetarian, Vegan]"


In [92]:
most_relevant_columns = ['organic_label', 'vegan_label']
search_query = "maggi noodles"

Best_results, running_time = SearchInputQuery(search_query, 50, index, 5, most_relevant_columns)

print(f"Execution time: {running_time*1e3} milli-seconds")
print (f"Search Query: {search_query} \n")
Best_results[['name','labels']]


Execution time: 28.512239456176758 milli-seconds
Search Query: maggi noodles 



,name,labels
6235,Alnatura Bio Gemüse Bouillon 6x 11G,"[Bio, Glutenfree, Vegan, Palmoel]"
3915,Rote-Linsen-Burger vegan (gekühlt),"[Bio, Vegetarian, Vegan]"
19749,Vegane Streichwurst,"[Bio, Vegan]"
20364,Bio Azuki-Soja Asian Spices Tasty Meal,[Bio]
19803,Bio Mungo-Edamame Thai Spices Tasty Meal,"[Bio, Glutenfree]"


In [81]:
most_relevant_columns = ['price']
search_query = "beef meat"

Best_results, running_time = SearchInputQuery(search_query, 50, index, 5, most_relevant_columns)

print(f"Execution time: {running_time*1e3} milli-seconds")
print (f"Search Query: {search_query} \n")
Best_results[['name','labels', 'price']]


Execution time: 23.720502853393555 milli-seconds
Search Query: beef meat 



,name,labels,price
20925,Salami Snack,"[Bio, Non-Vegetarian]",0.89
20538,Bio Salami Snack,"[Bio, Non-Vegetarian]",0.99
20352,Bio Rinder Salami Snack,"[Bio, Non-Vegetarian]",1.19
186,Schweinehackfleisch,[Non-Vegetarian],2.99
215,Dicke Sauerländer Bockwurst,[Non-Vegetarian],4.79
